<a href="https://colab.research.google.com/github/mikedaczar21/LLM_Research/blob/main/bootcamp/tutorials/quickstart/build_RAG_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/build_RAG_with_milvus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/build_RAG_with_milvus.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>

# Build RAG with Milvus

<img src="https://raw.githubusercontent.com/milvus-io/bootcamp/master/bootcamp/tutorials/quickstart/apps/rag_search_with_milvus/pics/rag_demo.png"/>

In this tutorial, we will show you how to build a RAG(Retrieval-Augmented Generation) pipeline with Milvus.

The RAG system combines a retrieval system with a generative model to generate new text based on a given prompt. The system first retrieves relevant documents from a corpus using Milvus, and then uses a generative model to generate new text based on the retrieved documents.


## Preparation
### Dependencies and Environment

In [ ]:
!pip install networkx torch_geometric scikit-learn pandas numpy matplotlib captum

In [ ]:
! pip install --upgrade pymilvus  requests tqdm

> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing libraries

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import torch
import networkx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import networkx as nx
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.utils import from_networkx
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
# ---- RUN PARAMETERS FOR GRAPH ATTENTION NETWORK
# PARAMETERS
target_column = 'SUR_TO_COMP_FLAG'
test_size = 0.2
val_size = 0.1
epochs = 80
learning_rate = 0.01
GAT_hidden_channels = 8
GAT_heads = 8
GAT_output_channels = 2 # Number of output classes in this case 2 (0 or 1 for the surrender to competitor flag)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

In [ ]:
# Loading data
current_data_path = "/content/drive/..../rs_oa_4Q24.csv"

current_data_df = pd.read_csv(current_data_path)

--------------- Helper Functions for feature transformations --------

In [ ]:

def extract_district(district_code):
    # Extracting substring after numeric portion
    if isinstance(district_code, str) == False:
        district_code = str(district_code)


    # Removes leading numeric/hyphen pattens
    extract_district = re.sub(r'^[\d]+\s*', '', district_code )

    # Only keeping alphabetical characters
    extract_district = re.sub(r'[^A-Za-z\s]', '', str(district_code) )

    words = [word for word in extract_district.split() if word]

    cleaned = " ".join(words)


In [ ]:
def sort_age_bracket(brackets):
    if isinstance(brackets, str) != True:
        brackets = str(brackets)

    def get_lower_bound(b):
        print(f"Brackets {b}")
        if isinstance(b, int):
            return b
        elif b.endswith('+'):
            return int(b[:1])

        b = str(b)
        # elif len(int(b)) == 0: return int(b)
        return int(b.split('-')[0])
    return sorted(brackets, key=get_lower_bound)

In [ ]:
age_bracket_order = {
    "0": 0,
    "0 - 30": 1,
    "30 - 39": 2,
    "40 - 49": 3,
    "50 - 59": 4,
    "60 - 64": 5,
    "65 - 69": 6,
      "70 - 79": 7,
      "80 +": 8

}

----- Performing transformations --------

In [ ]:
current_data_df['District_Extract'] = current_data_df['DISTRICT_CODE_NAME'].apply(extract_district)

In [ ]:
current_data_df["age_bracket_ordinal"] = current_data_df['AGE_BAND'].map(age_bracket_order)

In [ ]:
current_data_df['martial_status_cat'] = pd.Categorical(current_data_df['MARITAL_STATUS_MR'] ).codes
current_data_df['advisor_type_cat'] = pd.Categorical(current_data_df['ADVISOR_TYPE'] ).codes

-------- Setting up numeric columns and preparing labels/node features

In [ ]:
numeric_columns = [
     'martial_status_cat',
     'advisor_type_cat',
     "age_bracket_ordinal",
     'AGENT_TENURE',
     'INPLAN_ASSETS_CM',
       'OUTPLAN_ASSETS_CM',
       'AGT_ENG_LAST_12M',
        'REC_ENG_LAST_3M',
        'FLOWING_IND',
       'TOTAL_GPS_ASSETS',
       'GPS_FLAG'
]

In [ ]:
labels = torch.tensor(current_data_df['SUR_TO_COMP_FLAG'].values, dtype=torch.int)
node_features = torch.tensor(current_data_df[numeric_columns].values, dtype = torch.float)

In [ ]:
num_nodes = len(current_data_df)

# Building Graph using networkx


In [ ]:
# Build homogeneous graph using networkx

G = nx.Graph()
G.add_nodes_from(range(num_nodes))
print(f"Graph initialized with {G.number_of_nodes()} nodes.")

In [ ]:
# Adding nodes based on relationships

# Edge type 1: clients with similar advisors
advisor_groups = current_data_df.groupby('AGENT_CODE').groups
for group, clients in advisor_groups.items():
    clients = clients.to_list()
    for i in range(len(clients)):
        for j in range(i+1, len(clients)):

            # Checking if edge already exists to avoid duplicates
            if not G.has_edge(clients[i], clients[j]):
                G.add_edge(clients[i], clients[j], relationship='advisor_assigned')


# Edge type 2: clients in similar districts
district_groups = current_data_df.groupby('DISTRICT_CODE_NAME').groups
for group, clients in advisor_groups.items():
    clients = clients.to_list()
    for i in range(len(clients)):
        for j in range(i+1, len(clients)):

            if not G.has_edge(clients[i], clients[j]):
                G.add_edge(clients[i], clients[j], relationship='district')


# Edge type 3: Clients with similar inplan assets (top 10% similarity)
from sklearn.metrics.pairwise import cosine_similarity

threshold = np.percentile(assets_sim, 90)
assets_sim = cosine_similarity(current_data_df[ [ 'INPLAN_ASSETS_CM']])
np.fill_diagonal(asset_sim, 0) # Ignore self-similarity
similar_pairs = np.where(asset_sim > threshold)

for i, j in zip(*similar_pairs):
    # avoiding duplicates and self loops
    if i < j:
        if not G.has_edge(i, j):
            G.add_edge(i, j, relationship='asset_similarity')

In [ ]:
# print out graph information
print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges. \n Average node degree: { 2 * (G.number_of_edges() / G.number_of_nodes():.2f)}")

------ Preparing Pytorch Geometric data and creating train/test/validation splits

We will use OpenAI as the LLM in this example. You should prepare the [api key](https://platform.openai.com/docs/quickstart) `OPENAI_API_KEY` as an environment variable.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-***********"

### Prepare the data

We use the FAQ pages from the [Milvus Documentation 2.4.x](https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip) as the private knowledge in our RAG, which is a good data source for a simple RAG pipeline.

Download the zip file and extract documents to the folder `milvus_docs`.

In [ ]:
! wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
! unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

We load all markdown files from the folder `milvus_docs/en/faq`. For each document, we just simply use "# " to separate the content in the file, which can roughly separate the content of each main part of the markdown file.

In [ ]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

### Prepare the Embedding Model

We initialize the OpenAI client to prepare the embedding model.

In [ ]:
from openai import OpenAI

openai_client = OpenAI()

Define a function to generate text embeddings using OpenAI client. We use the [text-embedding-3-small](https://platform.openai.com/docs/guides/embeddings) model as an example.

In [ ]:
def emb_text(text):
    return (
        openai_client.embeddings.create(input=text, model="text-embedding-3-small")
        .data[0]
        .embedding
    )

Generate a test embedding and print its dimension and first few elements.

In [ ]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

1536
[0.00988506618887186, -0.005540902726352215, 0.0068014683201909065, -0.03810417652130127, -0.018254263326525688, -0.041231658309698105, -0.007651153020560741, 0.03220026567578316, 0.01892443746328354, 0.00010708322952268645]


## Load data into Milvus

### Create the Collection

In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

> As for the argument of `MilvusClient`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, you can set up a more performant Milvus server on [docker or kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server uri, e.g.`http://localhost:19530`, as your `uri`.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.

Check if the collection already exists and drop it if it does.

In [ ]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

Create a new collection with specified parameters.

If we don't specify any field information, Milvus will automatically create a default `id` field for primary key, and a `vector` field to store the vector data. A reserved JSON field is used to store non-schema-defined fields and their values.

In [ ]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

### Insert data
Iterate through the text lines, create embeddings, and then insert the data into Milvus.

Here is a new field `text`, which is a non-defined field in the collection schema. It will be automatically added to the reserved JSON dynamic field, which can be treated as a normal field at a high level.

In [ ]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 72/72 [00:27<00:00,  2.67it/s]


{'insert_count': 72,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71],
 'cost': 0}

## Build RAG

### Retrieve data for a query

Let's specify a frequent question about Milvus.

In [ ]:
question = "How is data stored in milvus?"

Search for the question in the collection and retrieve the semantic top-3 matches.

In [ ]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

Let's take a look at the search results of the query


In [ ]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.7883545756340027
    ],
    [
        "How does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float

### Use LLM to get a RAG response

Convert the retrieved documents into a string format.

In [ ]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

Define system and user prompts for the Lanage Model. This prompt is assembled with the retrieved documents from Milvus.

In [ ]:
SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

Use OpenAI ChatGPT to generate a response based on the prompts.

In [ ]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Milvus stores data in persistent storage as incremental logs, including inserted data (vector data, scalar data, and collection-specific schema) and metadata. Inserted data is stored in various object storage backends like MinIO, AWS S3, Google Cloud Storage, Azure Blob Storage, Alibaba Cloud OSS, and Tencent Cloud Object Storage. Metadata generated within Milvus is stored in etcd.


## Quick Deploy

To learn about how to start an online demo with this tutorial, please refer to [the example application](https://github.com/milvus-io/bootcamp/tree/master/bootcamp/tutorials/quickstart/apps/rag_search_with_milvus).